In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import utils.GenSnippetsLib as gsl
import PIL
import PIL.Image

In [2]:
df_fixation = pd.read_csv('./data/filteredData/fixation_stats.csv', sep=";")
df_fixation

,Participant,Algorithm,Behavioral,StartTime,EndTime,Duration,IsOutlier,SkillScore,Fixation_startT,Fixation_endT,Fixation_x,Fixation_y,Fixation_x_range,Fixation_y_range
0,1,IsPrime,Right,0.000000,12.390280,12.390280,False,0.331385,[ 0. 256.007 756.017 1136.027 1356...,[ 232.003 732.017 1112.027 1332.034 1512...,[ 938.53660583 1015.99482536 1111.40476227 90...,[455.2750361 458.47175181 454.98245269 482.13...,[0.47429751 1.03018639 1.93792793 0.55711697 0...,[1.20406041 1.38680553 0.92237319 0.94949848 0...
1,1,SiebDesEratosthenes,Wrong,49.778238,202.350152,152.571914,False,0.331385,[6.00050000e+01 2.40010000e+02 7.56025000e+02 ...,[ 212.011 732.025 1484.047 1932.068 ...,[ 868.43007088 1007.4270916 1058.09211731 109...,[291.24348164 299.36495841 298.56575131 298.83...,[ 0.48993109 1.25416388 1.31951268 1.368347...,[ 0.98835416 1.03392199 1.15453978 0.971170...
2,1,IsAnagram,Right,254.122970,363.738694,109.615724,False,0.331385,[5.59990000e+01 2.16007000e+02 8.16028000e+02 ...,[ 192.008 792.028 1004.036 1252.046 ...,[ 915.18871307 873.20179939 872.12508202 93...,[202.82147884 209.60132539 257.19715655 269.01...,[0.38803218 0.70631827 0.36245902 0.39378108 0...,[1.3295969 1.27731226 1.0016671 0.82503029 1...
3,1,RemoveDoubleChar,Right,401.629505,455.085781,53.456276,False,0.331385,[ 0. 144.003 316.009 724.024 956...,[ 128.001 292.01 708.023 932.031 1392...,[ 755.08060455 869.5891571 917.79919624 99...,[289.39472079 287.10810274 292.48779595 294.61...,[0.56010773 0.5157867 1.29034051 0.56324224 1...,[1.36399022 1.67141707 1.40613635 1.3199133 1...
4,1,BinToDecimal,Right,555.252914,605.175005,49.922091,False,0.331385,[ 0. 152.004 332.012 556.018 796...,[ 136.005 316.012 532.017 772.025 972...,[ 813.72073174 857.88272381 908.06817055 100...,[442.74862647 433.95372748 438.14722717 445.20...,[0.45957416 0.59362373 0.66204565 0.49548783 0...,[ 1.05953849 2.88387212 1.57175633 1.207385...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,71,DumpSorting,Wrong,2913.891198,3027.260143,113.368945,False,0.435651,[ 0. 148.003 476.015 860.038 ...,[1.12003000e+02 4.52015000e+02 8.28025000e+02 ...,[ 784.28146362 843.5760498 984.98113632 74...,[ 402.68189549 135.43147422 124.32098731 24...,[ 0.62816844 1.18571596 1.00710697 9.686770...,[1.46929439 1.10845543 0.97169054 2.8639374 1...
1068,71,BinomialCoefficient,Right,3138.411363,3189.049224,50.637861,False,0.435651,[ 0. 180.002 496.013 744.025 960...,[ 152.002 472.013 712.022 944.029 1168...,[ 898.8488102 995.05531311 911.90717697 80...,[380.1622349 376.28351927 381.3793999 458.69...,[0.43449658 0.37841624 0.51664131 0.3588181 0...,[0.73190452 0.94403384 1.16224789 1.15495182 0...
1069,71,IsAnagram,Right,3227.575716,3338.571470,110.995754,False,0.435651,[3.60000000e+01 2.60009000e+02 6.24025000e+02 ...,[ 232.009 604.025 812.03 1032.039 ...,[ 901.21260166 849.42790031 774.66410637 88...,[183.96801241 157.75504857 182.35525511 193.53...,[0.41719424 0.38097427 0.61861535 0.48578809 1...,[1.14784796 1.5037924 1.08456121 1.23995663 2...
1070,71,ArrayAverage,Right,3375.662381,3383.972645,8.310264,False,0.435651,[ 0. 176.006 884.026 1108.03 1336.04 ...,[ 152.006 868.024 1072.031 1312.039 2012.059 ...,[ 876.82525635 1010.44661522 884.08400059 93...,[337.34915793 341.4681378 445.61004728 499.84...,[0.42700017 1.7542686 0.42859525 0.46904707 1...,[1.41563445 2.79525118 0.9857216 1.93688978 2...


In [16]:
# Get Token based AOIs
snippets = df_fixation["Algorithm"].unique()
df_token_aois = pd.DataFrame(columns=["Algorithm", "Token", "TokenIdx", "BoundingBox"])
for snippet in snippets:
    aoi_token_generator = f"./../CodeSnippets/aois/Generators/{snippet}_ast.json"
    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="./../CodeSnippets/fonts/ttf/")
    aoi_clustered = []
    current_left = None
    current_top = None
    current_right = None
    current_bottom = None
    current_aoi = None
    color = None
    for letter in aoi_list:
        if len(letter["AOI"]) == 1 or letter["letter"] == '\n':
            if current_aoi is not None:
                aoi_clustered.append((len(aoi_clustered), current_aoi, current_left, current_top, current_right, current_bottom, color))
            current_aoi = None
            color = None
            current_left = None
            current_top = None
            current_right = None
            current_bottom = None
            continue
        if current_aoi is None:
            current_aoi = letter["AOI"][1]
            color = letter["color"]
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]
        elif current_aoi == letter["AOI"][1]:
            current_left = min(current_left, letter["BoundingBox"][0])
            current_top = min(current_top, letter["BoundingBox"][1])
            current_right = max(current_right, letter["BoundingBox"][2])
            current_bottom = max(current_bottom, letter["BoundingBox"][3])
        else:
            aoi_clustered.append((len(aoi_clustered), current_aoi, current_left, current_top, current_right, current_bottom, color))
            current_aoi = letter["AOI"][1]
            color = letter["color"]
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]

    for token in aoi_clustered:
        df_token_aois.loc[len(df_token_aois)] = [snippet, token[1], token[0], (token[2], token[3], token[4], token[5])]
df_token_aois

,Algorithm,Token,TokenIdx,BoundingBox
0,IsPrime,Modifier,0,"(45, 9, 93, 23)"
1,IsPrime,Modifier,1,"(101, 9, 149, 20)"
2,IsPrime,BasicType,2,"(157, 10, 213, 20)"
3,IsPrime,Identifier,3,"(221, 9, 277, 20)"
4,IsPrime,Separator,4,"(277, 8, 285, 22)"
...,...,...,...,...
2700,Rectangle,Keyword,82,"(253, 313, 285, 324)"
2701,Rectangle,Separator,83,"(285, 321, 293, 324)"
2702,Rectangle,Identifier,84,"(293, 313, 341, 327)"
2703,Rectangle,Separator,85,"(341, 312, 365, 327)"


In [ ]:
df_token_aois

In [ ]:
{'letter': ' ', 'color': (0, 0, 0, 255), 'bold': False, 'italic': False, 'AOI': ['None'], 'font': <PIL.ImageFont.FreeTypeFont object at 0x0000026D66B75D60>, 'BoundingBox': (5, 20, 13, 20)}